In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('orders_analysis').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 15:18:36 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.40 instead (on interface en0)
26/01/19 15:18:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 15:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
orders = spark.read.options(header=True, inferSchema=True, sep='|')\
    .csv('data/input/orders.csv')
    
orders.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|invoiceno|stockcode|         description|quantity|invoicedate|unitprice|customerid|       country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   544586|    21890|S/6 WOODEN SKITTL...|       3| 2011-02-21|     2.95|     17338|United Kingdom|
|   541104|   84509G|SET OF 4 FAIRY CA...|       3| 2011-01-13|     3.29|      NULL|United Kingdom|
|   560772|    22499|WOODEN UNION JACK...|       3| 2011-07-20|     4.96|      NULL|United Kingdom|
|   555150|    22488|NATURAL SLATE REC...|       5| 2011-05-31|     3.29|      NULL|United Kingdom|
|   570521|    21625|VINTAGE UNION JAC...|       3| 2011-10-11|     6.95|     12371|   Switzerland|
|   547053|    22087|PAPER BUNTING WHI...|      40| 2011-03-20|     2.55|     13001|United Kingdom|
|   573360|    22591|CARDHOLDER GINGHA...|       6| 2011-10-30|     3.25|     15748|United Kingdom|


In [4]:
result = orders.filter(~F.col('invoiceno').startswith('C'))

result.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|invoiceno|stockcode|         description|quantity|invoicedate|unitprice|customerid|       country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   544586|    21890|S/6 WOODEN SKITTL...|       3| 2011-02-21|     2.95|     17338|United Kingdom|
|   541104|   84509G|SET OF 4 FAIRY CA...|       3| 2011-01-13|     3.29|      NULL|United Kingdom|
|   560772|    22499|WOODEN UNION JACK...|       3| 2011-07-20|     4.96|      NULL|United Kingdom|
|   555150|    22488|NATURAL SLATE REC...|       5| 2011-05-31|     3.29|      NULL|United Kingdom|
|   570521|    21625|VINTAGE UNION JAC...|       3| 2011-10-11|     6.95|     12371|   Switzerland|
|   547053|    22087|PAPER BUNTING WHI...|      40| 2011-03-20|     2.55|     13001|United Kingdom|
|   573360|    22591|CARDHOLDER GINGHA...|       6| 2011-10-30|     3.25|     15748|United Kingdom|
